In [ ]:
!git clone https://github.com/ztuntrade/untrade-sdk.git
%cd untrade-sdk

Cloning into 'untrade-sdk'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 118 (delta 56), reused 95 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 43.67 KiB | 860.00 KiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/untrade-sdk


In [ ]:
!pip install .

Processing /content/untrade-sdk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5082 sha256=31197b3357f64b30ee36b40c8baf39aa49051fa7e417fb9f79f49806376b6a57
  Stored in directory: /root/.cache/pip/wheels/29/c8/c6/b5ec5185ff18cd2e956de7d1835f1e47ca6b9c115449fce17a
Successfully built untrade


In [ ]:
from untrade.client import Client
def perform_backtest(csv_file_path):
    client = Client()
    result = client.backtest(
        file_path=csv_file_path,
        leverage=1,
        jupyter_id="screening"
    )
    return result

if __name__ == "__main__":
    csv_file_path = r"/content/untrade-sdk/score_output_kalman_1h_long.csv"
    backtest_result = perform_backtest(csv_file_path)
    print(backtest_result)
    for value in backtest_result:
        print(value)

<generator object Client._handle_response_stream at 0x7b407d607300>
data: {
  "jupyter_id": "screening",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 00:00:00",
      "Total Trades": 257,
      "Leverage Applied": 1.0,
      "Winning Trades": 211,
      "Losing Trades": 46,
      "No. of Long Trades": 257,
      "No. of Short Trades": 0,
      "Benchmark Return(%)": 325.632937,
      "Benchmark Return(on $1000)": 3256.329373,
      "Win Rate": 82.101167,
      "Winning Streak": 24,
      "Losing Streak": 3,
      "Gross Profit": 6561.236029,
      "Net Profit": 6175.736029,
      "Average Profit": 24.030101,
      "Maximum Drawdown(%)": 5.411233,
      "Average Drawdown(%)": 0.126765,
      "Largest Win": 223.983759,
      "Average Win": 31.533723,
      "Largest Loss": -55.757594,
      "Average Loss": -10.388685,
      "Maximum Holding Time": "5 days 0:0:0",
      "Average Holding Time": "1 days 15:2:6"

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
print("imported!!")


imported!!


In [ ]:
def calculate_vwap(df):
    df['typical_price'] = (df['high'] + df['low'] + df['close']) / 3

    df['tp_volume'] = df['typical_price'] * df['volume']

    df['cumulative_tp_volume'] = df['tp_volume'].cumsum()
    df['cumulative_volume'] = df['volume'].cumsum()

    df['vwap'] = df['cumulative_tp_volume'] / df['cumulative_volume']

    return df


In [ ]:
df_30m = pd.read_csv(r"/content/drive/MyDrive/BTC_2019_2023_8h.csv")
window = 10
df_30m['rolling_mean'] = df_30m['close'].rolling(window=window).mean()
df_30m['rolling_std'] = df_30m['close'].rolling(window=window).std()
df_30m['z_score'] = (df_30m['close'] - df_30m['rolling_mean']) / df_30m['rolling_std']
calculate_vwap(df_30m)
#df_30m['kalman_trend'] = df_30m['close'].ewm(span=10).mean() # only for biggg datasets!!
df_30m.shape

(4727, 15)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pykalman

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.1/248.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
from pykalman import KalmanFilter
import numpy as np

close_prices = df_30m['close'].values
n = len(close_prices)

# Define fixed Kalman filter parameters to prevent forward bias
kf = KalmanFilter(
    transition_matrices=[1],  # Constant model
    observation_matrices=[1],  # Direct observation
    initial_state_mean=close_prices[0],  # Start at first price
    initial_state_covariance=1,  # Small initial variance
    transition_covariance=0.01,  # Low process noise
    observation_covariance=1  # Measurement noise
)

# Initialize arrays for results
filtered_means = np.zeros(n)

# Step-by-step filtering to avoid forward bias
for t in range(n):
    filtered_means[t], _ = kf.filter(close_prices[:t+1])  # Only use data up to time t

# Store the trend
df_30m['kalman_trend'] = filtered_means


## Long only algo (Bill Ackman style)

In [ ]:
#assigning scores
## ALgo strictly for longs!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#df_30m['scores'] = 0
# df_30m['scores'] += np.where(df_30m['close'] < df_30m['vwap'], 1, -1)  # VWAP
 # Z-Score
#df_30m['scores'] += np.where(df_30m['kalman_trend'] >= df_30m['close'], 1, -1)  # SMA Crossover
df_30m['scores'] = 0
df_30m['scores'] += 0.2 * np.where(df_30m['close'] < df_30m['vwap'], 1, -1)  # VWAP weight
df_30m['scores'] += 0.2 *np.where(df_30m['z_score'] < -1, 1, 0) + np.where(df_30m['z_score'] > 1, -1, 0)
df_30m['scores'] += 0.6 * np.where(df_30m['kalman_trend'] >= df_30m['close'], 1, -1)  # Kalman weight
#df_30m['signal'] = np.where(df_30m['score'] >= 2, 1, np.where(df_30m['score'] <= -2, -1, 'NEUTRAL'))
prev_trade_type = "Neutral"
# df_30m.at[0,'signals'] = 1
# df_30m.at[0,'trade_type'] = "Long"
for i in range(0,len(df_30m)):
    if(df_30m.at[i,'scores']>=0.4):
        if(prev_trade_type == "Long"):
            df_30m.at[i,'signals']=0
            df_30m.at[i,'trade_type'] = "Hold"

        if(prev_trade_type=="Neutral"):
            df_30m.at[i,'signals']=1
            df_30m.at[i,'trade_type'] = "Long"
            prev_trade_type="Long"

 #       if(prev_trade_type=="Short"):
#         if(prev_trade_type=="Short"):
#                 df_30m.at[i,'signals']=2
#                 df_30m.at[i,'trade_type'] = "Long"
#                 prev_trade_type="Long"


    if(df_30m.at[i,'scores']<=-0.4):
#         if(prev_trade_type == "Short"):
#             df_30m.at[i,'signals']=0
#             df_30m.at[i,'trade_type'] = "Hold"

        if(prev_trade_type=="Neutral"):
            df_30m.at[i,'signals']=0
            df_30m.at[i,'trade_type'] = "Neutral"
            prev_trade_type="Neutral"

        if(prev_trade_type=="Long"):
            df_30m.at[i,'signals']=-1
            df_30m.at[i,'trade_type'] = "Sold"
            prev_trade_type="Neutral"

    elif(df_30m.at[i,'scores']>-0.4 and df_30m.at[i,'scores']<0.4):
#         if(prev_trade_type == "Short"):
#             df_30m.at[i,'signals']=0
#             df_30m.at[i,'trade_type'] = "Hold"
#             prev_trade_type = "Neutral"

        if(prev_trade_type=="Neutral"):
            df_30m.at[i,'signals']=0
            df_30m.at[i,'trade_type'] = "Neutral"
            prev_trade_type="Neutral"

        if(prev_trade_type=="Long"):
            df_30m.at[i,'signals']=-1
            df_30m.at[i,'trade_type'] = "Hold"
            prev_trade_type="Neutral"

print("Trade Count",df_30m['scores'][(df_30m['scores']>=0.5)].count())

Trade Count 1543


## Long Short (Typical hedge fund style)

In [ ]:
#assigning scores
#df_30m['scores'] = 0
# df_30m['scores'] += np.where(df_30m['close'] < df_30m['vwap'], 1, -1)
 # Z-Score
#df_30m['scores'] += np.where(df_30m['kalman_trend'] >= df_30m['close'], 1, -1)
df_30m['scores'] = 0
df_30m['scores'] += 0.2 * np.where(df_30m['close'] < df_30m['vwap'], 1, -1)
df_30m['scores'] += 0.2 *np.where(df_30m['z_score'] < -1, 1, 0) + np.where(df_30m['z_score'] > 1, -1, 0)
df_30m['scores'] += 0.6 * np.where(df_30m['kalman_trend'] >= df_30m['close'], 1, -1)
#df_30m['signal'] = np.where(df_30m['score'] >= 2, 1, np.where(df_30m['score'] <= -2, -1, 'NEUTRAL'))
prev_trade_type = "Long"
df_30m.at[0,'signals'] = 1
df_30m.at[0,'trade_type'] = "Long"
for i in range(1,len(df_30m)):
    if(df_30m.at[i,'scores']>=0.3):
        if(prev_trade_type == "Long"):
            df_30m.at[i,'signals']=0
            df_30m.at[i,'trade_type'] = "Hold"

        if(prev_trade_type=="Neutral"):
            df_30m.at[i,'signals']=1
            df_30m.at[i,'trade_type'] = "Long"
            prev_trade_type="Long"

 #       if(prev_trade_type=="Short"):
        if(prev_trade_type=="Short"):
                df_30m.at[i,'signals']=2
                df_30m.at[i,'trade_type'] = "Long"
                prev_trade_type="Long"


    if(df_30m.at[i,'scores']<=-0.3):
        if(prev_trade_type == "Short"):
            df_30m.at[i,'signals']=0
            df_30m.at[i,'trade_type'] = "Hold"

        if(prev_trade_type=="Neutral"):
            df_30m.at[i,'signals']=-1
            df_30m.at[i,'trade_type'] = "Short"
            prev_trade_type="Short"

        if(prev_trade_type=="Long"):
            df_30m.at[i,'signals']=-2
            df_30m.at[i,'trade_type'] = "Short"
            prev_trade_type="Short"

    elif(df_30m.at[i,'scores']>-0.3 and df_30m.at[i,'scores']<0.3):
        if(prev_trade_type == "Short"):
            df_30m.at[i,'signals']=1
            df_30m.at[i,'trade_type'] = "Cover"
            prev_trade_type = "Neutral"

        if(prev_trade_type=="Neutral"):
            df_30m.at[i,'signals']=0
            df_30m.at[i,'trade_type'] = "Neutral"
            prev_trade_type="Neutral"

        if(prev_trade_type=="Long"):
            df_30m.at[i,'signals']=-1
            df_30m.at[i,'trade_type'] = "Hold"
            prev_trade_type="Neutral"

print("Trade Count",df_30m['scores'][(df_30m['scores']>=0.5)].count())

Trade Count 9998


In [ ]:
df_30m.to_csv('score_output_kalman_8h_long.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd

def check_forward_bias(df_30m, num_trials=10):
    mismatches = 0

    for _ in range(num_trials):
        random_idx = np.random.randint(10, len(df_30m) - 1)  # Pick a random row (avoid early points)
        original_signal = df_30m.at[random_idx, 'signals']  # Store the original signal

        truncated_df = df_30m.iloc[:random_idx].copy()  # Keep only past data
        prev_trade_type = "Neutral"

        # Recompute only `signals`
        truncated_df['scores'] = 0
        truncated_df['scores'] += 0.2 * np.where(truncated_df['close'] < truncated_df['vwap'], 1, -1)
        truncated_df['scores'] += 0.2 * np.where(truncated_df['z_score'] < -1, 1, 0) + np.where(truncated_df['z_score'] > 1, -1, 0)
        truncated_df['scores'] += 0.6 * np.where(truncated_df['kalman_trend'] >= truncated_df['close'], 1, -1)

        truncated_df['signals'] = 0  # Reset signals
        for i in range(len(truncated_df)):
            if truncated_df.at[i, 'scores'] >= 0.4:
                if prev_trade_type == "Neutral":
                    truncated_df.at[i, 'signals'] = 1
                    prev_trade_type = "Long"
                elif prev_trade_type == "Long":
                    truncated_df.at[i, 'signals'] = 0
            elif truncated_df.at[i, 'scores'] <= -0.4:
                if prev_trade_type == "Long":
                    truncated_df.at[i, 'signals'] = -1
                    prev_trade_type = "Neutral"
                else:
                    truncated_df.at[i, 'signals'] = 0
            else:
                if prev_trade_type == "Long":
                    truncated_df.at[i, 'signals'] = -1
                    prev_trade_type = "Neutral"
                else:
                    truncated_df.at[i, 'signals'] = 0

        # Compare computed signal at the random index
        new_signal = truncated_df.at[random_idx - 1, 'signals']  # Last computed signal
        if new_signal != original_signal:
            mismatches += 1

    print(f"Forward bias detected in {mismatches} out of {num_trials} trials.")

# Run the bias check
check_forward_bias(df_30m)



Forward bias detected in 4 out of 10 trials.
